**加载模型**

In [ ]:
from keras.utils import np_utils
from keras.optimizers import Adam
import numpy as np
import cv2
from keras import backend
from keras.layers import Dense, GlobalAveragePooling2D, BatchNormalization, Conv2D, Dropout
from keras.layers import *
import h5py
from tensorflow.python.keras.utils import conv_utils
from keras.preprocessing import image
from keras.models import Model, load_model
from keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from tqdm import tqdm
import math
from keras.initializers import glorot_uniform
from tensorflow.keras import regularizers
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
!wget https://digix-algo-challenge.obs.cn-east-2.myhuaweicloud.com/2020/cv/6rKDTsB6sX8A1O2DA2IAq7TgHPdSPxJF/test_data_A.zip

In [ ]:
!unzip test_data_A.zip 

In [ ]:
import torch
import torch.nn as nn
from torchvision import datasets, models, transforms
import torch.optim as optim
from torch.utils.data import DataLoader
import time 
from torch.utils.data import DataLoader, Sampler, Dataset
from torchvision import datasets,transforms
from torch.autograd import Variable

In [ ]:
from PIL import Image
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

**读取模型**

In [ ]:
# 首先构建不带分类器的预训练模型
base_model = ResNet50(weights='imagenet', include_top=False)
# 添加全局平均池化层
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
# 添加一个全连接层
x = Dense(4096, activation='relu')(x)
x = BatchNormalization()(x)
# 添加一个分类器，我们有3097个类
predictions = Dense(3097, activation='softmax')(x)
# 构建我们需要训练的完整模型
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
model.load_weights('/content/drive/Shared drives/my_new_pan/5_resnet50_finetuned4096bn.h5')

In [ ]:
import os
#path为gallery路径
def get_imlist(path):
    return [os.path.join(path, f) for f in os.listdir(path) if f.endswith('.jpg')]

In [ ]:
gallery = get_imlist('test_data_A/gallery')

In [ ]:
def prep_feat(path):
  img = image.load_img(path, target_size=(224, 224))
  img = image.img_to_array(img)
  img = np.expand_dims(img, axis=0)
  img = preprocess_input(img)
  return img

In [ ]:
def extractFeatures_fc(img_path, model):
  img = prep_feat(img_path)
  features = model.predict(img)
  return features

**Pytorch**

In [ ]:
model = torch.load('/content/drive/Shared drives/my_new_pan/10.pt')

In [ ]:
model.eval()

In [ ]:
import os
#path为gallery路径
def get_imlist(path):
    return [os.path.join(path, f) for f in os.listdir(path) if f.endswith('.jpg')]

In [ ]:
def prep_feat(path):
  img = Image.open(path)
  img = transforms.RandomResizedCrop(224)(img)
  img = transforms.ToTensor()(img)
  if img.shape[0] == 1:
    img = img.numpy()
    img = np.concatenate((img, img, img))
    img = torch.from_numpy(img)               
  img = transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])(img)
  img = torch.unsqueeze(img, dim=0)
  return img.cuda()

In [ ]:
from tqdm import tqdm
import math

In [ ]:
gallery = get_imlist('test_data_A/gallery')

In [ ]:
def extractFeatures_fc(img_path, model):
  img = prep_feat(img_path)
  features = model(img)
  return features.cpu().detach().numpy()

**未训练**

In [ ]:
model = ResNet50(weights='imagenet')

In [ ]:
for layer in model.layers:
  print(layer.name)

In [ ]:
model = Model(inputs=model.input, outputs=model.get_layer('conv5_block2_out').output)

In [ ]:
import os
#path为gallery路径
def get_imlist(path):
    return [os.path.join(path, f) for f in os.listdir(path) if f.endswith('.jpg')]

In [ ]:
def prep_feat(path):
  img = image.load_img(path, target_size=(224, 224))
  img = image.img_to_array(img)
  img = np.expand_dims(img, axis=0)
  img = preprocess_input(img)
  return img

In [ ]:
gallery = get_imlist('test_data_A/gallery')

In [ ]:
def extractFeatures_fc(img_path, model):
  img = prep_feat(img_path)
  features = model.predict(img)
  return features

**https://github.com/fmaglia/keras_rmac_plus**

In [ ]:
def calculateMAC(features): #max-pooling and l2-norm
  mac = []
  rows = features.shape[1] * features.shape[2]
  cols = features.shape[3]
  features1 = np.reshape(features, (rows, cols))
  features2 = np.amax(features1, axis = 0)
  features2 /= np.linalg.norm(features2, 2)
  mac.append(features2)
  return mac

In [ ]:
def calculateRMAC(features, L):
  RMAC = []
  W = features.shape[1]
  H = features.shape[2]
  for l in range(1,L+1):
    if (l==1):
      heightRegion = widthRegion = min(W,H)
      if (W<H):
        xRegions = 1
        yRegions = 2
      else:
        xRegions = 2
        yRegions = 1
    else:
      widthRegion = heightRegion = math.ceil(2*min(W,H)/(l+1))
      if (l==2):
        xRegions = 2
        yRegions = 3
      elif (l==3):
        xRegions = 3
        yRegions = 2

    if (widthRegion * xRegions < W):
        #not covered the image along width
      widthRegion = math.ceil(W/xRegions)

    if (heightRegion*yRegions < H):
	    heightRegion = math.ceil(H/yRegions)
    
    coefW = W / xRegions
    coefH = H / yRegions 

		# print("L:",l," w:",widthRegion," h:",heightRegion,"xRegions",xRegions,"yRegions",yRegions)
    for x in range(0,xRegions):
      for y in range(0,yRegions):
        initialX = round(x*coefW)
        initialY = round(y*coefH)
        finalX = initialX + widthRegion
        finalY = initialY + heightRegion
        if (finalX > W):
          finalX = W
          initialX = finalX - widthRegion
        if (finalY > H):
          finalY = H
          initialY =  finalY - heightRegion

				# print(" X ",initialX,":", finalX," Y ", initialY,":", finalY)

        featureRegion = features[:,initialX:finalX,initialY:finalY,:] #(old implementation)
        RMAC.append(calculateMAC(featureRegion))
  return RMAC

In [ ]:
def extractFeatures_rmac(img_path, model, RMAC=True, L = 3):
	img = prep_feat(img_path)
	features = model.predict(img)
	result = calculateMAC(features)
	if (RMAC):
		result = calculateRMAC(features, L)
	return result

In [ ]:
def get_feature_map_rmac():
  features = []
  img_names = []
  for i, img_path in tqdm(enumerate(gallery)):
    feature = extractFeatures_rmac(img_path,model)
    img_name = os.path.split(img_path)[1]
    feature /= np.linalg.norm(feature)
    features.append(feature)
    img_names.append(img_name)
  features = np.array(features)
  features = np.squeeze(features)
  return features, img_names

In [ ]:
def get_feature_map_fc():
  features = []
  img_names = []
  for i, img_path in tqdm(enumerate(gallery)):
    feature = extractFeatures_fc(img_path,model)
    img_name = os.path.split(img_path)[1]
    feature /= np.linalg.norm(feature)
    features.append(feature)
    img_names.append(img_name)
  features = np.array(features)
  features = np.squeeze(features)
  return features, img_names

In [ ]:
rmac_features, rmac_img_names = get_feature_map_rmac()

49804it [50:34, 16.41it/s]


In [ ]:
h5f = h5py.File('/content/drive/Shared drives/my_new_pan/4096-bn-rmac', 'w')
h5f.create_dataset('dataset_1', data=rmac_features)
h5f.create_dataset('dataset_2', data=np.string_(rmac_img_names))
h5f.close()

In [ ]:
fc_features, fc_img_names = get_feature_map_fc()

49804it [24:00, 34.57it/s]


In [ ]:
h5f = h5py.File('/content/drive/Shared drives/my_new_pan/4096-bn-rmac', 'w')
h5f.create_dataset('dataset_1', data=fc_features)
h5f.create_dataset('dataset_2', data=np.string_(fc_img_names))
h5f.close()

In [ ]:
query = get_imlist('test_data_A/query')

In [ ]:
len(query)

9600

In [ ]:
def get_query_feature_map_rmac():
  #first step inception
  features = []
  img_names = []
  for i, img_path in tqdm(enumerate(query)):
    feature = extractFeatures_rmac(img_path,model)
    feature = np.squeeze(feature)
    img_name = os.path.split(img_path)[1]
    feature /= np.linalg.norm(feature)
    features.append(feature)
    img_names.append(img_name)
  features = np.array(features)
  features = np.squeeze(features)
  return features, img_names

In [ ]:
query_features, query_img_names = get_query_feature_map_rmac()

9600it [10:07, 15.80it/s]


In [ ]:
h5f = h5py.File('/content/drive/Shared drives/my_new_pan/4096-bn-rmac_query', 'w')
h5f.create_dataset('dataset_1', data=query_features
h5f.create_dataset('dataset_2', data=np.string_(query_img_names)))
h5f.close()

**特征文件在以上已经存储完毕，在下方完成加载，开始计算**

**欧式距离**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import h5py
import numpy as np
import pandas as pd
from tqdm import tqdm

In [ ]:
h5f = h5py.File('/content/drive/Shared drives/my_new_pan/4096-bn-rmac', 'r')
feature = h5f['dataset_1'][:]
imgNames = h5f['dataset_2'][:]
h5f.close()

In [ ]:
h5f = h5py.File('/content/drive/Shared drives/my_new_pan/4096-bn-rmac_query', 'r')
queryfeature = h5f['dataset_1'][:]
queryimgNames = h5f['dataset_2'][:]
h5f.close()

In [ ]:
retype_imgNames = [i.decode('utf-8') for i in imgNames]
retype_queryimgNames = [i.decode('utf-8') for i in queryimgNames]

In [ ]:
submission = []
for i in tqdm(range(len(queryimgNames))):
  query_img_name = retype_queryimgNames[i]
  query_feature = queryfeature[i]
  score = []
  for feat in feature:
    score.append(np.linalg.norm(query_feature-feat))
  rank_ID = np.argsort(score)
  imlist = []

  for index in rank_ID[0:10]:
    imlist.append(retype_imgNames[index])

  submission.append([query_img_name,str(imlist).replace('\"','').replace('\'','').replace(' ','').replace('[','{').replace(']','}')])
 
submission = np.array(submission)


100%|██████████| 9600/9600 [3:14:07<00:00,  1.21s/it]


In [ ]:
pd.DataFrame(submission).to_csv("/content/drive/Shared drives/my_new_pan/submission.csv", header=None, index=None)

**余弦相似度**

In [ ]:
submission = []
for i in tqdm(range(len(queryimgNames))):
  query_img_name = retype_queryimgNames[i]
  query_feature = queryfeature[i]
  score = []
  for feat in feature:
    cos = np.dot(feat,query_feature.T)/np.linalg.norm(feat)/np.linalg.norm(query_feature)
    sin = 0.5 + 0.5 * cos
    score.append(sin)
  rank_ID = np.argsort(score)[::-1]
  imlist = []
  for index in rank_ID[0:10]:
    imlist.append(retype_imgNames[index])
  submission.append([query_img_name,str(imlist).replace('\"','').replace('\'','').replace(' ','').replace('[','{').replace(']','}')])

submission = np.array(submission)

100%|██████████| 9600/9600 [2:40:34<00:00,  1.00s/it]


In [ ]:
import pandas as pd

In [ ]:
pd.DataFrame(submission).to_csv("/content/drive/Shared drives/my_new_pan/submission.csv", header=None, index=None)

格式修整

In [ ]:
with open("/content/drive/Shared drives/my_new_pan/submission.csv") as f:
  contents = f.readlines()

newcontents=[]
for i in contents:
  i = i.replace('\"','')
  i = i.replace(' ','')
  newcontents.append(i)

In [ ]:
with open("/content/drive/Shared drives/my_new_pan/submission2.csv",'w') as f:
  f.writelines(newcontents)